In [13]:
from fuzzywuzzy import fuzz
import pandas as pd
import math
import json

In [123]:
df_warehouse_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/movie.csv")
df_movielen_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/preprocess/movie.csv")
df_metacritic_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/preprocess/movie.csv")
df_tmdb_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/tmdb/preprocess/movie.csv")

In [124]:
# Biến đổi các trường có kiểu string list về dạng list phục vụ cho những biến đổi về sau
def string_to_list(string):
    if type(string)==str:
        return string.split(", ")
    else:
        return string
df_metacritic_movie["genre"]=[string_to_list(i) for i in df_metacritic_movie["genre"]]
df_metacritic_movie["director"]=[string_to_list(i) for i in df_metacritic_movie["director"]]
df_metacritic_movie["actor"]=[string_to_list(i) for i in df_metacritic_movie["actor"]]
df_movielen_movie["genres"]=[string_to_list(i) for i in df_movielen_movie["genres"]]
df_tmdb_movie["genres"]=[string_to_list(i) for i in df_tmdb_movie["genres"]]
df_tmdb_movie["cast"]=[string_to_list(i) for i in df_tmdb_movie["cast"]]
df_tmdb_movie["crew"]=[string_to_list(i) for i in df_tmdb_movie["crew"]]
df_warehouse_movie["genres"]=[string_to_list(i) for i in df_warehouse_movie["genres"]]
df_warehouse_movie["directors"]=[string_to_list(i) for i in df_warehouse_movie["directors"]]
df_warehouse_movie["actors"]=[string_to_list(i) for i in df_warehouse_movie["actors"]]

In [125]:
print("Số bộ phim không rõ năm sản xuất:")
print("Movielen: "+str(sum(df_movielen_movie['year'].isna())))
print("Metacritic: "+str(sum(df_metacritic_movie['year'].isna())))
print("TMDB: "+str(sum(df_tmdb_movie['release_date'].isna())))
print("Warehouse: "+str(sum(df_warehouse_movie['release_date'].isna())))


Số bộ phim không rõ năm sản xuất:
Movielen: 410
Metacritic: 0
TMDB: 0
Warehouse: 1166


Ta sẽ data matching movie nguồn và movie trong data warehouse theo title và year, cụ thể nếu xét một bộ phim trong nguồn A:
- Ta sẽ lọc trong data warehouse những phim nào có cùng năm sản xuất với phim trong nguồn A + những phim nào mà năm sản xuất không xác định => xác định tập ứng cử viên
- Sau khi có tập ứng cử viên, ta sẽ sử dụng string matching để tìm ra phim trong data warehouse có tên match với tên phim trong nguồn A (với threshold 90)
- Đối với những phim không rõ năm sản xuất => ta sẽ lấy toàn bộ phim trong data warehouse làm candidate, tuy nhiên trong quá trình string matching ta sẽ đặt threshold cao hơn (95), lý do là vì trong cùng một năm khả năng ra được những bộ phim có tên trùng nhau hoặc na ná nhau là rất thấp

In [111]:
def title_matching(df_candidate, warehouse_id_field, warehouse_title_field, warehouse_year_field, 
                   source_title, source_year, threshold1=94, threshold2=90):
    result=[]
    for i in df_candidate.index:
        if math.isnan(source_year) or math.isnan(df_candidate[warehouse_year_field][i]):
            threshold = threshold1
        else:
            threshold = threshold2
        matching=fuzz.token_sort_ratio(df_candidate[warehouse_title_field][i], source_title)
        if matching>=threshold:
            result.append([df_candidate[warehouse_id_field][i], df_candidate[warehouse_title_field][i], 
                           df_candidate[warehouse_year_field][i], matching])
            break
    if len(result)==0:
        return [None, None, None, None]
    else:
        return result[0]
def movie_matching(source_df, warehouse_df, 
                   source_id_field, source_title_field, source_year_field,
                   warehouse_id_field, warehouse_title_field, warehouse_year_field):
    source_movie_id = []
    source_movie_title = []
    source_movie_year = []            
    warehouse_movie_id = []
    warehouse_movie_title = []
    warehouse_movie_year = []
    matching = []
    for i in source_df.index:
        if i%1000==0:
            print(i)
        source_title = source_df[source_title_field][i]
        source_year = source_df[source_year_field][i]
        source_movie_id.append(source_df[source_id_field][i])
        source_movie_title.append(source_title)
        source_movie_year.append(source_year)
        if math.isnan(source_year):
            df_candidate = warehouse_df[[warehouse_id_field, warehouse_title_field, warehouse_year_field]]
            result = title_matching(df_candidate, warehouse_id_field, warehouse_title_field, warehouse_year_field, 
                                    source_title, source_year)
            warehouse_movie_id.append(result[0])
            warehouse_movie_title.append(result[1])
            warehouse_movie_year.append(result[2])
            matching.append(result[3])
        else:
            df_candidate = warehouse_df[(warehouse_df[warehouse_year_field]==source_year)|
                                        (warehouse_df[warehouse_year_field].isna())]
            df_candidate = df_candidate[[warehouse_id_field, warehouse_title_field, warehouse_year_field]]
            result = title_matching(df_candidate, warehouse_id_field, warehouse_title_field, warehouse_year_field, 
                                    source_title, source_year)
            warehouse_movie_id.append(result[0])
            warehouse_movie_title.append(result[1])
            warehouse_movie_year.append(result[2])
            matching.append(result[3])
    return pd.DataFrame(list(zip(source_movie_id, warehouse_movie_id, source_movie_title, warehouse_movie_title, 
                                 matching, source_movie_year, warehouse_movie_year)),
                        columns =['source_movie_id', 'warehouse_movie_id', 'source_movie_title', 'warehouse_movie_title',
                                    'matching', 'source_movie_year', 'warehouse_movie_year'])
    

Giải thích tham số của các hàm:
- movie_matching:
    + source_df: dataframe của dữ liệu nguồn ngoài (VD: df_movielen_movie)
    + warehouse_df: dataframe của dữ liệu warehouse (VD: df_warehouse_movie)
    + source_id_field: tên trường movie id của dữ liệu nguồn ngoài (VD: movieId trong df_movielen_movie)
    + source_title_field: tên trường movie title của dữ liệu nguồn ngoài (VD: title trong df_movielen_movie)
    + source_year_field: tên trường movie year của dữ liệu nguồn ngoài (VD: year trong df_movielen_movie)
    + warehouse_id_field: tên trường movie id của dữ liệu warehouse (VD: movie_id trong df_warehouse_movie)
    + warehouse_title_field: tên trường movie title của dữ liệu warehouse (VD: movie_title trong df_warehouse_movie)
    + warehouse_year_field: tên trường movie year của dữ liệu warehouse (VD: year trong df_warehouse_movie)
- title_matching:

# Movielen

## Data Matching

In [55]:
movielen_result =  movie_matching(source_df=df_movielen_movie, warehouse_df=df_warehouse_movie, 
                                  source_id_field='movieId', source_title_field='title', source_year_field='year',
                                  warehouse_id_field='movie_id', warehouse_title_field='movie_title', warehouse_year_field='release_date')
movielen_result

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000


,source_movie_id,warehouse_movie_id,source_movie_title,warehouse_movie_title,matching,source_movie_year,warehouse_movie_year
0,1,movie16405,Toy Story,Toy Story,100.0,1995.0,1995.0
1,2,movie1063,Jumanji,Jumanji,100.0,1995.0,1995.0
2,3,movie7342,Grumpier Old Men,Grumpier Old Men,100.0,1995.0,1995.0
3,4,movie16982,Waiting to Exhale,Waiting to Exhale,100.0,1995.0,1995.0
4,5,movie6344,Father of the Bride Part II,Father of the Bride: Part II,100.0,1995.0,1995.0
...,...,...,...,...,...,...,...
62418,209157,None,We,None,NaN,2018.0,NaN
62419,209159,None,Window of the Soul,None,NaN,2001.0,NaN
62420,209163,None,Bad Poems,None,NaN,2018.0,NaN
62421,209169,None,A Girl Thing,None,NaN,2001.0,NaN


In [59]:
movielen_result_final = movielen_result[movielen_result['warehouse_movie_id'].notna()]
movielen_result_final

,source_movie_id,warehouse_movie_id,source_movie_title,warehouse_movie_title,matching,source_movie_year,warehouse_movie_year
0,1,movie16405,Toy Story,Toy Story,100.0,1995.0,1995.0
1,2,movie1063,Jumanji,Jumanji,100.0,1995.0,1995.0
2,3,movie7342,Grumpier Old Men,Grumpier Old Men,100.0,1995.0,1995.0
3,4,movie16982,Waiting to Exhale,Waiting to Exhale,100.0,1995.0,1995.0
4,5,movie6344,Father of the Bride Part II,Father of the Bride: Part II,100.0,1995.0,1995.0
...,...,...,...,...,...,...,...
62325,208751,movie13941,Strange Bedfellows,Strange Bedfellows,100.0,2004.0,NaN
62330,208773,movie8107,I Lost My Body,I Lost My Body,100.0,2019.0,2019.0
62352,208843,movie2713,An Acceptable Loss,An Acceptable Loss,100.0,2019.0,2019.0
62374,208939,movie9063,Klaus,Klaus,100.0,2019.0,2019.0


In [58]:
movielen_result_final.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/data_matching/movielen_matching.csv")

## Bổ sung dữ liệu đối với những bộ phim đã có trong data warehouse

In [126]:
# Mapping id của movielen và id của movie warehouse
warehouse_movielen_mapping = {}
for i in movielen_result_final.index:
    warehouse_movielen_mapping[movielen_result_final["warehouse_movie_id"][i]]=movielen_result_final["source_movie_id"][i]

In [127]:
# Hàm update những trường mang giá trị dạng value kiểu float
def update_float_value_field(index, warehouse_df, warehouse_id_field, warehouse_update_field, 
                source_df, source_id_field, source_update_field, warehouse_source_mapping):
    warehouse_id = warehouse_df[warehouse_id_field][index]
    warehouse_field = warehouse_df[warehouse_update_field][index]
    if warehouse_id in warehouse_source_mapping:
        # Chỉ khi nào thông tin trong data warehouse bị trống mà trong nguồn dữ liệu ngoài lại có thông tin đó
        # thì ta mới tiến hành update
        if math.isnan(warehouse_field):
            source_field = source_df[source_df[source_id_field]==warehouse_source_mapping[warehouse_id]][source_update_field]
            if not math.isnan(source_field):
                return source_field
        else:
            return warehouse_field
    else:
        return warehouse_field

# Hàm update những trường mang giá trị dạng value kiểu string
def update_string_value_field(index, warehouse_df, warehouse_id_field, warehouse_update_field, 
                source_df, source_id_field, source_update_field, warehouse_source_mapping):
    warehouse_id = warehouse_df[warehouse_id_field][index]
    warehouse_field = warehouse_df[warehouse_update_field][index]
    if warehouse_id in warehouse_source_mapping:
        # Chỉ khi nào thông tin trong data warehouse bị trống mà trong nguồn dữ liệu ngoài lại có thông tin đó
        # thì ta mới tiến hành update
        if type(warehouse_field)==float:
            source_field = source_df[source_df[source_id_field]==warehouse_source_mapping[warehouse_id]][source_update_field]
            if type(source_field)!=float:
                return source_field
        else:
            return warehouse_field
    else:
        return warehouse_field

# Hàm update những trường mang giá trị dạng list
def update_list_field(index, warehouse_df, warehouse_id_field, warehouse_update_field, 
                source_df, source_id_field, source_update_field, warehouse_source_mapping):
    warehouse_id = warehouse_df[warehouse_id_field][index]
    warehouse_field = warehouse_df[warehouse_update_field][index]
    if warehouse_id in warehouse_source_mapping:
        source_field = source_df[source_df[source_id_field]==warehouse_source_mapping[warehouse_id]][source_update_field]
        if type(warehouse_field)==float:
            if type(source_field)==list:
                return source_field
            else:
                return None
        else:
            if type(source_field)==list:
                return list(set(source_field+warehouse_field))
            else:
                return warehouse_field
    else:
        return warehouse_field

In [128]:
df_warehouse_movie["release_date"] = [update_float_value_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="release_date",
                                                  source_df=df_movielen_movie, source_id_field="movieId", 
                                                  source_update_field="year", warehouse_source_mapping=warehouse_movielen_mapping)
                                      for i in df_warehouse_movie.index]

In [129]:
df_warehouse_movie["genres"] = [update_list_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="genres",
                                                  source_df=df_movielen_movie, source_id_field="movieId", 
                                                  source_update_field="genres", warehouse_source_mapping=warehouse_movielen_mapping)
                                      for i in df_warehouse_movie.index]

## Thêm những bộ phim mới chưa có trong data warehouse

In [130]:
# Lấy ra list các mã bộ phim không có trong warehouse của movielen và tạo các movie id mới cho warehouse ứng với các phim trên
# Trong bài này ta sẽ chỉ lấy 1000 phim để tránh bị crash khi chạy trên vs code
remain_id_movielen = [i for i in df_movielen_movie["movieId"].unique() if i not in warehouse_movielen_mapping.values()][:1000]
count=len(df_warehouse_movie)
for i in remain_id_movielen:
    warehouse_movielen_mapping["movie"+str(count)]=i
    count+=1
movielen_warehouse_mapping = {y: x for x, y in warehouse_movielen_mapping.items()}
with open("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/data_matching/movielen_warehouse_id_mapping.json", "w") as outfile:
    json.dump({str(y): x for x, y in warehouse_movielen_mapping.items()}, outfile)

In [131]:
# Thêm những phim mới trong movielen vào warehouse
for i in remain_id_movielen:
    df_warehouse_movie = df_warehouse_movie.append({"movie_id":movielen_warehouse_mapping[i],
                               "movie_title":df_movielen_movie[df_movielen_movie['movieId']==i]['title'],
                               "movie_info":None,
                               "genres": df_movielen_movie[df_movielen_movie['movieId']==i]['genres'],
                               "directors":None,
                               "actors":None,
                               "release_date":df_movielen_movie[df_movielen_movie['movieId']==i]['year'],
                               "runtime":None,
                               "content_rating":None
                               }, ignore_index=True)

/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/2828085599.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":movielen_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/2828085599.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":movielen_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/2828085599.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":movielen_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/2828085599.py:

## Xử lý dữ liệu rating và review

In [132]:
df_movielen_rating = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/preprocess/movielen_rating.csv")
df_movielen_review = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/movielen/preprocess/movielen_review.csv")

df_movielen_rating = df_movielen_rating[df_movielen_rating['movieId'].isin(list(movielen_warehouse_mapping.keys()))]
df_movielen_review = df_movielen_review[df_movielen_review['movieId'].isin(list(movielen_warehouse_mapping.keys()))]

In [133]:
# Đổi giá trị movieId cho phù hợp với data warehouse
df_movielen_rating['movieId']=[movielen_warehouse_mapping[i] for i in df_movielen_rating['movieId']]
df_movielen_review['movieId']=[movielen_warehouse_mapping[i] for i in df_movielen_review['movieId']]

In [134]:
print("df_movielen_rating fields: " + str(df_movielen_rating.columns))
print("df_movielen_review fields: " + str(df_movielen_review.columns))

df_movielen_rating fields: Index(['movieId', 'avgrating', 'ratecount'], dtype='object')
df_movielen_review fields: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [135]:
# Đổi tên các trường sao cho phù hợp với data warehouse
df_movielen_rating.rename(columns = {'movieId':'movie_id'}, inplace = True)
df_movielen_review.rename(columns = {'movieId':'movie_id', 'timestamp': 'review_date', 'rating': 'review_score'}, inplace = True)

In [136]:
df_movielen_rating.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/movielen_rating.csv", index=False)
df_movielen_review.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/movielen_review.csv", index=False)

# Metacritic

## Data Matching

In [137]:
# Chuyển kiểu của trường release date sang float để tránh bị lỗi
df_warehouse_movie=df_warehouse_movie.astype({"release_date": float})

In [138]:
metacritic_result =  movie_matching(source_df=df_metacritic_movie, warehouse_df=df_warehouse_movie, 
                                  source_id_field='movie_id', source_title_field='title', source_year_field='year',
                                  warehouse_id_field='movie_id', warehouse_title_field='movie_title', warehouse_year_field='release_date')
metacritic_result

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


,source_movie_id,warehouse_movie_id,source_movie_title,warehouse_movie_title,matching,source_movie_year,warehouse_movie_year
0,meta0,movie4617,Citizen Kane,Citizen Kane,100.0,1941.0,1941.0
1,meta1,movie7091,The Godfather,The Godfather,100.0,1972.0,1972.0
2,meta2,movie686,Rear Window,Rear Window,100.0,1954.0,1954.0
3,meta3,None,Casablanca,None,NaN,1943.0,NaN
4,meta4,movie3940,Boyhood,Boyhood,100.0,2014.0,2014.0
...,...,...,...,...,...,...,...
14208,meta14208,None,The Garbage Pail Kids Movie,None,NaN,1987.0,NaN
14209,meta14209,movie16734,United Passions,United Passions,100.0,2015.0,2015.0
14210,meta14210,movie3601,Bio-Dome,Bio-Dome,100.0,1996.0,1996.0
14211,meta14211,None,Chaos,None,NaN,2005.0,NaN


In [139]:
metacritic_result_final = metacritic_result[metacritic_result['warehouse_movie_id'].notna()]
metacritic_result_final

,source_movie_id,warehouse_movie_id,source_movie_title,warehouse_movie_title,matching,source_movie_year,warehouse_movie_year
0,meta0,movie4617,Citizen Kane,Citizen Kane,100.0,1941.0,1941.0
1,meta1,movie7091,The Godfather,The Godfather,100.0,1972.0,1972.0
2,meta2,movie686,Rear Window,Rear Window,100.0,1954.0,1954.0
4,meta4,movie3940,Boyhood,Boyhood,100.0,2014.0,2014.0
6,meta6,movie16870,Vertigo,Vertigo,100.0,1958.0,1958.0
...,...,...,...,...,...,...,...
14204,meta14204,movie16958,Vulgar,Vulgar,100.0,2002.0,2002.0
14206,meta14206,movie7760,Hillary's America: The Secret History of the D...,Hillary's America: The Secret History of the D...,100.0,2016.0,2016.0
14207,meta14207,movie15809,The Singing Forest,The Singing Forest,100.0,2003.0,2003.0
14209,meta14209,movie16734,United Passions,United Passions,100.0,2015.0,2015.0


In [140]:
metacritic_result_final.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/data_matching/metacritic_matching.csv")

## Bổ sung dữ liệu đối với những bộ phim đã có trong data warehouse

In [141]:
# Mapping id của metacritic và id của movie warehouse
warehouse_metacritic_mapping = {}
for i in metacritic_result_final.index:
    warehouse_metacritic_mapping[metacritic_result_final["warehouse_movie_id"][i]]=metacritic_result_final["source_movie_id"][i]

In [142]:
df_warehouse_movie["release_date"] = [update_float_value_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="release_date",
                                                  source_df=df_metacritic_movie, source_id_field="movie_id", 
                                                  source_update_field="year", warehouse_source_mapping=warehouse_metacritic_mapping)
                                      for i in df_warehouse_movie.index]
df_warehouse_movie["content_rating"] = [update_string_value_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="content_rating",
                                                  source_df=df_metacritic_movie, source_id_field="movie_id", 
                                                  source_update_field="age_rating", warehouse_source_mapping=warehouse_metacritic_mapping)
                                      for i in df_warehouse_movie.index]
df_warehouse_movie["runtime"] = [update_float_value_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="runtime",
                                                  source_df=df_metacritic_movie, source_id_field="movie_id", 
                                                  source_update_field="runtime", warehouse_source_mapping=warehouse_metacritic_mapping)
                                      for i in df_warehouse_movie.index]
df_warehouse_movie["movie_info"] = [update_string_value_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="movie_info",
                                                  source_df=df_metacritic_movie, source_id_field="movie_id", 
                                                  source_update_field="description", warehouse_source_mapping=warehouse_metacritic_mapping)
                                      for i in df_warehouse_movie.index]

In [143]:
df_warehouse_movie["genres"] = [update_list_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="genres",
                                                  source_df=df_metacritic_movie, source_id_field="movie_id", 
                                                  source_update_field="genre", warehouse_source_mapping=warehouse_metacritic_mapping)
                                      for i in df_warehouse_movie.index]
df_warehouse_movie["directors"] = [update_list_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="directors",
                                                  source_df=df_metacritic_movie, source_id_field="movie_id", 
                                                  source_update_field="director", warehouse_source_mapping=warehouse_metacritic_mapping)
                                      for i in df_warehouse_movie.index]
df_warehouse_movie["actors"] = [update_list_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="actors",
                                                  source_df=df_metacritic_movie, source_id_field="movie_id", 
                                                  source_update_field="actor", warehouse_source_mapping=warehouse_metacritic_mapping)
                                      for i in df_warehouse_movie.index]

## Thêm những bộ phim mới chưa có trong data warehouse

In [144]:
# Lấy ra list các mã bộ phim không có trong warehouse của metacritic và tạo các movie id mới cho warehouse ứng với các phim trên
remain_id_metacritic = [i for i in df_metacritic_movie["movie_id"].unique() if i not in warehouse_metacritic_mapping.values()]
count=len(df_warehouse_movie)
for i in remain_id_metacritic:
    warehouse_metacritic_mapping["movie"+str(count)]=i
    count+=1
metacritic_warehouse_mapping = {y: x for x, y in warehouse_metacritic_mapping.items()}
with open("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/data_matching/metacritic_warehouse_id_mapping.json", "w") as outfile:
    json.dump({str(y): x for x, y in warehouse_metacritic_mapping.items()}, outfile)

In [145]:
# Thêm những phim mới trong metacritic vào warehouse
for i in remain_id_metacritic:
    df_warehouse_movie = df_warehouse_movie.append({"movie_id":metacritic_warehouse_mapping[i],
                               "movie_title":df_metacritic_movie[df_metacritic_movie['movie_id']==i]['title'],
                               "movie_info":df_metacritic_movie[df_metacritic_movie['movie_id']==i]['description'],
                               "genres": df_metacritic_movie[df_metacritic_movie['movie_id']==i]['genre'],
                               "directors":df_metacritic_movie[df_metacritic_movie['movie_id']==i]['director'],
                               "actors":df_metacritic_movie[df_metacritic_movie['movie_id']==i]['actor'],
                               "release_date":df_metacritic_movie[df_metacritic_movie['movie_id']==i]['year'],
                               "runtime":df_metacritic_movie[df_metacritic_movie['movie_id']==i]['runtime'],
                               "content_rating":df_metacritic_movie[df_metacritic_movie['movie_id']==i]['age_rating'],
                               }, ignore_index=True)

/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/1938971426.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":metacritic_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/1938971426.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":metacritic_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/1938971426.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":metacritic_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/19389714

## Xử lý dữ liệu rating và review

In [146]:
df_metacritic_rating = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/preprocess/metacritic_rating.csv")
df_metacritic_review = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/preprocess/metacritic_review.csv",  lineterminator='\n')

df_metacritic_rating = df_metacritic_rating[df_metacritic_rating['movie_id'].isin(list(metacritic_warehouse_mapping.keys()))]
df_metacritic_review = df_metacritic_review[df_metacritic_review['movie_id'].isin(list(metacritic_warehouse_mapping.keys()))]

In [147]:
# Đổi giá trị movieId cho phù hợp với data warehouse
df_metacritic_rating['movie_id']=[metacritic_warehouse_mapping[i] for i in df_metacritic_rating['movie_id']]
df_metacritic_review['movie_id']=[metacritic_warehouse_mapping[i] for i in df_metacritic_review['movie_id']]

In [165]:
print("df_metacritic_rating fields: " + str(df_metacritic_rating.columns))
print("df_metacritic_review fields: " + str(df_metacritic_review.columns))

df_metacritic_rating fields: Index(['movie_id', 'rating', 'img', 'url', 'ratecount'], dtype='object')
df_metacritic_review fields: Index(['user', 'type', 'grade', 'review', 'spoilers', 'language', 'movie_id'], dtype='object')


In [167]:
# Đổi tên các trường sao cho phù hợp với data warehouse
df_metacritic_rating.rename(columns = {'rating':'avgrating', 'url': 'link'}, inplace = True)
df_metacritic_review.rename(columns = {'grade': 'review_score', 'review': 'review_content', 'user': 'critic_name'}, inplace = True)

In [168]:
df_metacritic_rating.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/metacritic_rating.csv", index=False)
df_metacritic_review.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/metacritic_review.csv", index=False)

# TMDB

## Data Matching

In [150]:
# Chuyển kiểu của trường release date sang float để tránh bị lỗi
df_warehouse_movie=df_warehouse_movie.astype({"release_date": float})

In [151]:
tmdb_result =  movie_matching(source_df=df_tmdb_movie, warehouse_df=df_warehouse_movie, 
                                  source_id_field='id', source_title_field='title', source_year_field='release_date',
                                  warehouse_id_field='movie_id', warehouse_title_field='movie_title', warehouse_year_field='release_date')
tmdb_result

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


,source_movie_id,warehouse_movie_id,source_movie_title,warehouse_movie_title,matching,source_movie_year,warehouse_movie_year
0,19404,None,Dilwale Dulhania Le Jayenge,None,NaN,1995.0,NaN
1,278,movie13183,The Shawshank Redemption,The Shawshank Redemption,100.0,1994.0,1994.0
2,238,movie7091,The Godfather,The Godfather,100.0,1972.0,1972.0
3,724089,None,Gabriel's Inferno Part II,None,NaN,2020.0,NaN
4,424,movie12935,Schindler's List,Schindler's List,100.0,1993.0,1993.0
...,...,...,...,...,...,...,...
9475,21435,None,French Fried Vacation 3,None,NaN,2006.0,NaN
9476,17711,movie2365,The Adventures of Rocky & Bullwinkle,The Adventures of Rocky & Bullwinkle,100.0,2000.0,2000.0
9477,17532,None,S. Darko,None,NaN,2009.0,NaN
9478,13908,movie10170,The Master of Disguise,The Master of Disguise,100.0,2002.0,2002.0


In [152]:
tmdb_result_final = tmdb_result[tmdb_result['warehouse_movie_id'].notna()]
tmdb_result_final

,source_movie_id,warehouse_movie_id,source_movie_title,warehouse_movie_title,matching,source_movie_year,warehouse_movie_year
1,278,movie13183,The Shawshank Redemption,The Shawshank Redemption,100.0,1994.0,1994.0
2,238,movie7091,The Godfather,The Godfather,100.0,1972.0,1972.0
4,424,movie12935,Schindler's List,Schindler's List,100.0,1993.0,1993.0
6,240,movie7092,The Godfather: Part II,"The Godfather, Part II",100.0,1974.0,1974.0
15,497,movie7298,The Green Mile,The Green Mile,100.0,1999.0,1999.0
...,...,...,...,...,...,...,...
9472,19766,movie8400,Inspector Gadget 2,Inspector Gadget 2,100.0,2003.0,2003.0
9473,22345,movie3065,Baby Geniuses,Baby Geniuses,100.0,1999.0,1999.0
9476,17711,movie2365,The Adventures of Rocky & Bullwinkle,The Adventures of Rocky & Bullwinkle,100.0,2000.0,2000.0
9478,13908,movie10170,The Master of Disguise,The Master of Disguise,100.0,2002.0,2002.0


In [153]:
tmdb_result_final.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/data_matching/tmdb_matching.csv")

## Bổ sung dữ liệu đối với những bộ phim đã có trong data warehouse

In [154]:
# Mapping id của tmdb và id của movie warehouse
warehouse_tmdb_mapping = {}
for i in tmdb_result_final.index:
    warehouse_tmdb_mapping[tmdb_result_final["warehouse_movie_id"][i]]=tmdb_result_final["source_movie_id"][i]

In [156]:
df_warehouse_movie["release_date"] = [update_float_value_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="release_date",
                                                  source_df=df_tmdb_movie, source_id_field="id", 
                                                  source_update_field="release_date", warehouse_source_mapping=warehouse_tmdb_mapping)
                                      for i in df_warehouse_movie.index]
df_warehouse_movie["movie_info"] = [update_string_value_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="movie_info",
                                                  source_df=df_tmdb_movie, source_id_field="id", 
                                                  source_update_field="overview", warehouse_source_mapping=warehouse_tmdb_mapping)
                                      for i in df_warehouse_movie.index]

In [157]:
df_warehouse_movie["genres"] = [update_list_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="genres",
                                                  source_df=df_tmdb_movie, source_id_field="id", 
                                                  source_update_field="genres", warehouse_source_mapping=warehouse_tmdb_mapping)
                                      for i in df_warehouse_movie.index]
df_warehouse_movie["directors"] = [update_list_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="directors",
                                                  source_df=df_tmdb_movie, source_id_field="id", 
                                                  source_update_field="crew", warehouse_source_mapping=warehouse_tmdb_mapping)
                                      for i in df_warehouse_movie.index]
df_warehouse_movie["actors"] = [update_list_field(index=i, warehouse_df=df_warehouse_movie, 
                                                  warehouse_id_field="movie_id", warehouse_update_field="actors",
                                                  source_df=df_tmdb_movie, source_id_field="id", 
                                                  source_update_field="cast", warehouse_source_mapping=warehouse_tmdb_mapping)
                                      for i in df_warehouse_movie.index]

## Thêm những bộ phim mới chưa có vào trong data warehouse

In [158]:
# Lấy ra list các mã bộ phim không có trong warehouse của tmdb và tạo các movie id mới cho warehouse ứng với các phim trên
remain_id_tmdb = [i for i in df_tmdb_movie["id"].unique() if i not in warehouse_tmdb_mapping.values()]
count=len(df_warehouse_movie)
for i in remain_id_tmdb:
    warehouse_tmdb_mapping["movie"+str(count)]=i
    count+=1
tmdb_warehouse_mapping = {y: x for x, y in warehouse_tmdb_mapping.items()}
with open("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/data_matching/tmdb_warehouse_id_mapping.json", "w") as outfile:
    json.dump({str(y): x for x, y in warehouse_tmdb_mapping.items()}, outfile)

In [161]:
# Thêm những phim mới trong tmdb vào warehouse
for i in remain_id_tmdb:
    df_warehouse_movie = df_warehouse_movie.append({"movie_id":tmdb_warehouse_mapping[i],
                               "movie_title":df_tmdb_movie[df_tmdb_movie['id']==i]['title'],
                               "movie_info":df_tmdb_movie[df_tmdb_movie['id']==i]['overview'],
                               "genres": df_tmdb_movie[df_tmdb_movie['id']==i]['genres'],
                               "directors":df_tmdb_movie[df_tmdb_movie['id']==i]['crew'],
                               "actors":df_tmdb_movie[df_tmdb_movie['id']==i]['cast'],
                               "release_date":df_tmdb_movie[df_tmdb_movie['id']==i]['release_date'],
                               "runtime":None,
                               "content_rating":None,
                               }, ignore_index=True)

/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/179430684.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":tmdb_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/179430684.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":tmdb_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/179430684.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse_movie = df_warehouse_movie.append({"movie_id":tmdb_warehouse_mapping[i],
/var/folders/l1/4m9tz6cd01s9y33zg3gj6pxh0000gn/T/ipykernel_19826/179430684.py:3: FutureWarning

## Xử lý dữ liệu rating

In [162]:
df_tmdb_rating = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/tmdb/preprocess/tmdb_rating.csv")

df_tmdb_rating = df_tmdb_rating[df_tmdb_rating['id'].isin(list(tmdb_warehouse_mapping.keys()))]

In [163]:
# Đổi giá trị movieId cho phù hợp với data warehouse
df_tmdb_rating['id']=[tmdb_warehouse_mapping[i] for i in df_tmdb_rating['id']]

In [164]:
print("df_tmdb_rating fields: " + str(df_tmdb_rating.columns))

df_tmdb_rating fields: Index(['id', 'vote_average', 'vote_count'], dtype='object')


In [172]:
df_tmdb_rating.rename(columns = {'id':'movie_id', 'vote_average': 'avgrate', 'vote_count': 'ratecount'}, inplace = True)

In [173]:
df_tmdb_rating.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/tmdb_rating.csv", index=False)